<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/cifar10_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense,Activation, Flatten, Dropout,BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
import numpy as np
from matplotlib import pyplot

In [2]:
def normalize_images(items):
  temp = np.zeros(shape=(items.shape))
  mean = np.mean(items,axis=(0,1,2,3))
  std = np.std(items,axis=(0,1,2,3))
  temp =(items - mean) /(std +1e-7)
  return temp

In [3]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [5]:
(x_train,x_valid) = x_train[5000:],x_train[:5000]
(y_train, y_valid) = y_train[5000:],y_train[:5000]

In [6]:
# Training, Test, and validation data split:
print('x_train = ',x_train.shape)
print('x_valid = ',x_valid.shape)
print('x_test = ',x_test.shape)

x_train =  (45000, 32, 32, 3)
x_valid =  (5000, 32, 32, 3)
x_test =  (10000, 32, 32, 3)


In [7]:
# Data Normalization:
x_train = normalize_images(x_train)
x_valid = normalize_images(x_valid)
x_test = normalize_images(x_test)

In [8]:
# One-Hot Encocding of lables:
y_train = np_utils.to_categorical(y_train,10)
y_valid = np_utils.to_categorical(y_valid,10)
y_test = np_utils.to_categorical(y_test,10)

In [9]:
# Data Augmentation:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    vertical_flip=False
)

datagen.fit(x_train)

In [10]:
# Mode Variables:
base_hidden_units = 32
weight_decay= 1e-4

# Model Architecture Sequentional API based:
model = Sequential()

# 1st Convolution:
model.add(Conv2D(base_hidden_units,kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay), 
                 input_shape=x_train.shape[1:]))
# Relu is our main activation function
model.add(Activation('relu'))
# Batch Normalization will be applied to each Convolutional layer
model.add(BatchNormalization()) 

# 2nd Convolution:
model.add(Conv2D(base_hidden_units,kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.L2(weight_decay)))
# Activation:
model.add(Activation('relu'))
# Batch Normalization:
model.add(BatchNormalization())
# Pool & Dropout:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 3rd Convolution:
model.add(Conv2D(base_hidden_units*2, kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.L2(weight_decay)))
# Activation:
model.add(Activation('relu'))
# Batch Normalization:
model.add(BatchNormalization())

# 4rd Convolution:
model.add(Conv2D(base_hidden_units * 2,kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.L2(weight_decay)))
# Activation
model.add(Activation('relu'))
# Batch Normaliation:
model.add(BatchNormalization())
# Pool & Dropout:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# 5th Convolution:
model.add(Conv2D(base_hidden_units * 4,kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.L2(weight_decay)))
# Activation
model.add(Activation('relu'))
# Batch Normaliation:
model.add(BatchNormalization())

# 6th Convolution:
model.add(Conv2D(base_hidden_units * 4,kernel_size=3,padding='same',
                 kernel_regularizer=regularizers.L2(weight_decay)))
# Activation
model.add(Activation('relu'))
# Batch Normaliation:
model.add(BatchNormalization())
# Pool & Dropout:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Final Fully Connected Layer:
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [14]:
batch_size = 128
epochs = 200

checkpointer = ModelCheckpoint(filepath='model.100epochs.hdfs',verbose=1,save_best_only=True)
optimizer = keras.optimizers.Adam(learning_rate=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size)
                              ,callbacks=[checkpointer],steps_per_epoch=x_train.shape[0]//batch_size,
                              epochs=epochs,verbose=2,validation_data=(x_valid,y_valid))


Epoch 1/200
351/351 - 20s - loss: 0.3923 - accuracy: 0.8732 - val_loss: 0.5018 - val_accuracy: 0.8520

Epoch 00001: val_loss improved from inf to 0.50177, saving model to model.100epochs.hdfs
INFO:tensorflow:Assets written to: model.100epochs.hdfs/assets
Epoch 2/200
351/351 - 17s - loss: 0.3961 - accuracy: 0.8732 - val_loss: 0.5046 - val_accuracy: 0.8504

Epoch 00002: val_loss did not improve from 0.50177
Epoch 3/200
351/351 - 17s - loss: 0.3915 - accuracy: 0.8744 - val_loss: 0.4902 - val_accuracy: 0.8542

Epoch 00003: val_loss improved from 0.50177 to 0.49020, saving model to model.100epochs.hdfs
INFO:tensorflow:Assets written to: model.100epochs.hdfs/assets
Epoch 4/200
351/351 - 17s - loss: 0.3904 - accuracy: 0.8738 - val_loss: 0.4952 - val_accuracy: 0.8548

Epoch 00004: val_loss did not improve from 0.49020
Epoch 5/200
351/351 - 17s - loss: 0.3881 - accuracy: 0.8773 - val_loss: 0.4850 - val_accuracy: 0.8564

Epoch 00005: val_loss improved from 0.49020 to 0.48505, saving model to mod

In [15]:
scores = model.evaluate(x_test,y_test,batch_size=128,verbose=1)

79/79 [==============================] - 1s 7ms/step - loss: 0.5278 - accuracy: 0.8643


In [16]:
print('\n Validation Accuracy:: %.3f'%(scores[1]*100))


 Validation Accuracy:: 86.430
